In [9]:
{
    "name":"User information extractor",
    "description":"Extract useful information like (name,email,phone,location,age)",
    "parameters":{
        "type":"object",
        "properties":{
          "name":{
              "type":"String",
              "description":"Name of the user.",
          },
          "email":{
              "type":"String",
              "description":"Mail-id of the user",
              "pattern":"^[^@\\s]+@[^@\\s]+\\.[^@\\s]+$",
          },
          "phone":{
              "type":"String",
              "description":"Phone number of the user",
              "pattern":"^[0-9+\\-()\\s]{7,20}$",
          },
          "loaction":{
              "type":"String",
              "description":"Loaction of the user.",
          },
          "age":{
              "type":"Integer",
              "description":"Age of the user",
              "min":0,
              "max":100,
          },
        },
        "required":["name","email","phone","location","age"],
    }
}

{'name': 'User information extractor',
 'description': 'Extract useful information like (name,email,phone,location,age)',
 'parameters': {'type': 'object',
  'properties': {'name': {'type': 'String',
    'description': 'Name of the user.'},
   'email': {'type': 'String',
    'description': 'Mail-id of the user',
    'pattern': '^[^@\\s]+@[^@\\s]+\\.[^@\\s]+$'},
   'phone': {'type': 'String',
    'description': 'Phone number of the user',
    'pattern': '^[0-9+\\-()\\s]{7,20}$'},
   'loaction': {'type': 'String', 'description': 'Loaction of the user.'},
   'age': {'type': 'Integer',
    'description': 'Age of the user',
    'min': 0,
    'max': 100}},
  'required': ['name', 'email', 'phone', 'location', 'age']}}

In [19]:
import json 
from groq import Groq
from jsonschema import validate,ValidationError
import os 
from dotenv import load_dotenv
load_dotenv()

client = Groq(api_key=os.getenv("Task1\.env\GROK_API_KEY"))

schema={
    "type":"object",
   "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "pattern": "^[^@\\s]+@[^@\\s]+\\.[^@\\s]+$"},
        "phone": {"type": "string", "pattern": "^[0-9+\\-()\\s]{7,20}$"},
        "location": {"type": "string"},
        "age": {"type": "integer", "minimum": 0, "maximum": 100}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

def extract_user_information(chats:str):
  response=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {
              "role":"System","content":"Extract personal information from user chats",
          },
          {
              "role":"user","content":chats,
          },
      
      ],
      functions=[{
          "name":"User information extractor",
          "description":"Extract useful information like (name,email,phone,location,age)",
          "parameters":schema,
      }],
      function_call={"name":"User information extractor"},
      )
  data=json.loads(response.choices[0].message.function_call.arguments)
  return data


user_chats= [
    "Hi, my name is Khush godani. I’m 21 years old, live in Bangalore. You can contact me at khushgodani123@example.com or +91-9876543210.",
    "Hello, I’m Harshi Jogani, aged 21, staying in Mumbai. Reach me at harshijogani123@gmail.com or call at 8355929530.",
    "This is Disha Jain, 21 years old from New York. Email: dishajain123@outlook.com, phone: (123) 456-7890."
]

for i in user_chats :
  extracted = extract_user_information(i)
  try:
      validate(instance=extracted, schema=schema)
      print("Valid:", extracted)
  except ValidationError as e:
      print("Validation Error:", e.message)


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}